In [1]:
import numpy as np 
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import sys, os 
sys.path.append(os.path.abspath(os.path.join('../scripts/')))
# Import utility scripts
from data_cleaner import *

In [4]:
util = Clean_Data()

Utility Functions Imported!!!


In [5]:
def get_date_features(df,col):
    df['Year'] = df[col].dt.year
    df['Month'] = df[col].dt.month
    df['Day'] = df[col].dt.day
    df.drop(columns=[col], axis=1,inplace=True)

In [6]:
store_db = util.read_data('../data/store.csv')
train_db = util.read_data('../data/train.csv')

In [31]:
db = pd.merge(left=train_db,right=store_db,on='Store',how='inner').head(3000)

In [38]:
y = db[['Sales']]

In [32]:
X = db.drop(columns=['CompetitionOpenSinceMonth','CompetitionOpenSinceYear','Sales','Customers','PromoInterval'])

In [33]:
util.to_datetime(X,'Date','%Y-%m-%d')

In [34]:
get_date_features(X,'Date')

# Splitting to test and train sets

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Preprocessor

In [20]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [16]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.to_list()
categorical_features = X.select_dtypes(include=['object']).columns.to_list()

In [35]:
def encode(df,columns):
    df = pd.get_dummies(df,columns=columns)
    return df

In [36]:
X = encode(X,categorical_features)

In [40]:
numeric_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='constant', fill_value=0))
      ,('scaler', StandardScaler())
])


In [41]:
preprocessor = ColumnTransformer(
   transformers=[
    ('numeric', numeric_transformer, numeric_features)
]) 

# Estimator

In [44]:
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
pipeline = Pipeline(steps = [
               ('preprocessor', preprocessor)
              ,('regressor',RandomForestRegressor())
           ])

In [45]:
rf_model = pipeline.fit(X_train, y_train)
print (rf_model)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numeric',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=0,
                                                                                 missing_values=nan,
                                                                                 strategy='constant',
                                                          

# Model Accuracy

In [49]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
predictions = rf_model.predict(X_test)
print (f'r2_score : {r2_score(y_test, predictions)}\n\
RMSE:{np.sqrt(mean_squared_error(y_test, predictions))} \n\
MAE:{mean_absolute_error(y_test,predictions)} ')



r2_score : 0.9443512392585331
RMSE:693.7723029034814 
MAE:440.58736666666664 
